In [1]:
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
from math import pi, log
import pandas as pd

In [108]:
def entropy(i):
    j = np.where(np.in1d(atoms_ref.ids.tolist(), i)==True)[0].tolist()
    if len(j)>1:
        H = 0.5*len(j)*(1+log(2*pi))+0.5*np.linalg.slogdet(np.take(np.take(COV, j, axis = 0), j, axis = -1))[1]
        return H
    elif len(j)==1:
        H = 0.5*len(j)*(1+log(2*pi))+0.5*log(np.take(np.take(COV, j, axis = -1), j, axis = 0))
        return H
    else:
        return print('Wrong index format or no elements in the list were found')

def inf(i, j):
    if len(i)>=1 and len(j)>=1:
        Hi = entropy(i)
        Hj = entropy(j)
        Hij = entropy(np.unique(np.hstack((i, j))))
        return abs(Hi + Hj - Hij)
    else:
        print('Wrong index format or no elements in the list were found')

def mutual(COM):
    
    length = len(COM)
    mutual_out = np.empty((length, length))
    for i in range(length):
        for j in range(length):
            if i==j:
                mutual_out[i, j] = entropy(COM[i].ids.tolist())
            elif i< j:
                s1 = np.setdiff1d(COM[i].ids.tolist(), COM[j].ids.tolist(), assume_unique=True)                
                s2 = np.setdiff1d(COM[j].ids.tolist(), COM[i].ids.tolist(), assume_unique=True)                
                mutual_out[i, j] = inf(s1, s2)
            else:
                mutual_out[i, j] = 'NaN'
    return mutual_out


def correlation(i, ii):
    sum_H = 0
    for r in np.unique(ii.resindices):
        sum_H += entropy(ii.indices[np.where(ii.resindices==r)[0].tolist()])
    H = entropy(i)
    return sum_H - H

def conditional_entropy(i, j):
    k = np.setdiff1d(i, j, assume_unique=True)
    Hij = entropy(np.hstack((k, j)))
    Hj = entropy(j)
    return Hij - Hj

def conditional_total_correlation(i, ii, j):
    i = np.setdiff1d(i, j, assume_unique=True).tolist()
    sum_H = 0
    for r in np.unique(ii.resindices):
        t = ii.ids[np.where(ii.resindices==r)[0]].tolist()
        if False not in np.in1d(t, i):
            sum_H += conditional_entropy(t, j)
    H = conditional_entropy(i, j)
    return sum_H - H

def coordination_information(i, ii, j, jj):
    return (correlation(i, ii) - conditional_total_correlation(i, ii, j))

def normalized_cooordination_information(i, ii, j, jj):
    return coordination_information(i, ii, j, jj)/correlation(i, ii)

## COORDINATION

def coordination(COM):
    length = len(COM)
    coordination_output = np.empty((length, length))
    for i in range(length):
        for j in range(length):
            if i == j:
                coordination_output[i, j] = correlation(COM[i].ids.tolist(), COM[i])
            else:
                s1 = np.setdiff1d(COM[i].ids.tolist(), COM[j].ids.tolist(), assume_unique=True)               
                s2 = np.setdiff1d(COM[j].ids.tolist(), COM[i].ids.tolist(), assume_unique=True)                
                s11 = COM[i].ids.tolist()
                s22 = COM[j].ids.tolist()
                if s1.shape[0]*s2.shape[0]==0:
                    coordination_output[i, j] = 'NaN'
                else:
                    coordination_output[i, j] = normalized_cooordination_information(s11, COM[i], s22, COM[j])  
    return coordination_output

## Individual contribution

def contribution(A):
    all_contrib_output = []
    for i in np.unique(A.resindices):
        all_contrib_output.append(1-conditional_total_correlation(A.ids.tolist(), A, A.ids[np.where(A.resindices==i)[0].tolist()])/
        correlation(A.ids.tolist(), A))
    return all_contrib_output
    
def conditional_entropy(i, j):
    i = np.setdiff1d(i, j, assume_unique=True)
    Hij = entropy(np.hstack((i, j)))
    Hj = entropy(j)
    return Hij - Hj

def conditional_information(i, j, k):
    i = np.setdiff1d(i, k, assume_unique=True)
    j = np.setdiff1d(j, k, assume_unique=True)
    Hi = conditional_entropy(i, k)
    Hj = conditional_entropy(j, k)
    Hij = conditional_entropy(np.hstack((i, j)), k)
    return Hi + Hj - Hij

 
## Mutual contribution

def average_contribution_mutual(A, B):
    all_contrib_mutual = []
    sum_n_res = np.unique(np.hstack((A.resindices, B.resindices)))
    for k in sum_n_res:
        if k in A.resindices:
            res = A.ids[np.where(A.resindices==k)[0].tolist()]    
        elif k in B.resindices:
            res = B.ids[np.where(B.resindices==k)[0].tolist()]    
        all_contrib_mutual.append(1-conditional_information(A.ids.tolist(), B.ids.tolist(), res)
                             /inf(A.ids.tolist(), B.ids.tolist()))
    return all_contrib_mutual

def conditional_coordination_information(i, ii, j, k):
    return conditional_total_correlation(i, ii, k) - conditional_total_correlation(i, ii, np.hstack((j, k)))

## Contribution coordination

def average_contribution_coordination(A, B):
    sum_n_res = np.unique(np.hstack((A.resindices, B.resindices)))
    all_contrib_coord = np.empty((sum_n_res.shape[0]))
    for k in range(sum_n_res.shape[0]):
        n = sum_n_res[k]
        if n in A.resindices:
            res = A.ids[np.where(A.resindices==n)[0].tolist()] 
        elif n in B.resindices:
            res = B.ids[np.where(B.resindices==n)[0].tolist()] 
        all_contrib_coord[k] = 1 - conditional_coordination_information(A.ids.tolist(), A, B.ids.tolist(), res)/coordination_information(A.ids.tolist(), A, B.ids.tolist(), B)
    return all_contrib_coord

def co_information(i, j, k):
    part = conditional_information(i, j, k)
    total = inf(i, j)
    return total - part

## Mutual Channel

def mutual_channel(A, B):
    score = []
    for i in np.unique(atoms.resindices):
        score.append(co_information(A.ids.tolist(), B.ids.tolist(), atoms.ids[np.where(atoms.resindices==(i))[0].tolist()]))
    for a in np.unique(np.hstack((A.resindices, B.resindices))):
        score[a-1] = 0
    return np.asarray(score)
    
def mutual_coordination_information(i, ii, j, jj, k):
    return coordination_information(i, ii, j, jj) - conditional_coordination_information(i, ii, j, k)

## Coordination channel

def coordination_channel(A, B):
    score = []
    for i in np.unique(atoms.resindices):
        score.append(mutual_coordination_information(A.ids.tolist(), A, B.ids.tolist(), B, atoms.ids[np.where(atoms.resindices==i)[0].tolist()]))
    return np.asarray(score)

def compute_cov_matrix(atoms, atoms_ref, nsteps):
    p0 = atoms_ref.positions
    p0.shape

    # Compute the distance matrices
    N = u.trajectory.n_frames//nsteps
    dist_matrix = np.empty((p0.shape[0], nsteps, N))
    for n in range(N):
        for ts in u.trajectory[n*nsteps:(n+1)*nsteps]:
            dist_matrix[:, ts.frame - nsteps*n, n] = np.linalg.norm(atoms.positions-p0, axis = -1)[:]


    # Compute covariance matrix
    cov1 = np.empty((p0.shape[0], p0.shape[0], N))
    for n in range(N):
        cov1 [:, :, n] = np.cov(dist_matrix[:, :, n], rowvar=True)

    # Compute the averaged covariance matrix
    COV = np.average(cov1, axis = -1)
    
    return COV

## Pandas representation

def high_diff(s):
    return ['background-color: pink' if v> 0.10 else 'background-color: lightblue' if v<-0.10 else '' for v in s]
def high(s):
    return ['background-color: pink' if v > 0.2 else '' for v in s]


# The NbIT functions

mutual(COMB) - mutual information for all the defined sites

coordination(COMB) - normalized coordiantion for the entire set

contribution(COMB[i]) - individual contribution

average_contribution_coordination(COMB[i], COMB[j])

mutual_channel(COMB[i], COMB[j])

coordination_channel(COMB[i], COMB[j])

In [91]:
# Load the trajectory assuming it is already aligned (see align.ipynb)
u = mda.Universe('/Volumes/OWC_12TB_HD/LeuT_test/LeuT/cut.psf',
                '/Volumes/OWC_12TB_HD/LeuT_test/LeuT/fixed.xtc', 
                '/Volumes/OWC_12TB_HD/LeuT_test/LeuT/LeuT_nbit_aligned_cut.xtc')
# Load the reference structure
u_ref = mda.Universe('/Volumes/OWC_12TB_HD/LeuT_test/LeuT/cut.psf', 
                     '/Volumes/OWC_12TB_HD/LeuT_test/LeuT/cut.pdb')

In [92]:
# Select the regions for NbIT
S1 = u.select_atoms('(segid LIG and resid 519) or (segid PROT and resid 25 26 104 108 253 254 256 259 355 359) or (segid INT and resid 517) and not type H')
NA1 = u.select_atoms('(segid INT and resid 517) or (segid PROT and resid 22 27 254 286) or (segid LIG and resid 519) and not type H')
NA2 = u.select_atoms('(segid INT and resid 516) or (segid PROT and resid 20 23 351 354 355) and not type H')
S2 = u.select_atoms('(segid PROT and resid 29 30 107 111 114 253 319 320 324 400 404) and not type H')
INI = u.select_atoms('(segid PROT and resid 5 187 267 268 361 369) and not type H')

In [94]:
# Select combinations of regions for the analysis
COMB = [S1, S2, NA1, NA2, NA1+NA2, NA1+NA2+S1, NA1+NA2+S1+S2, INI]
names = ['S1', 'S2', 'NA1', 'NA2', 'NA1+NA2', 'NA1+NA2+S1', 'NA1+NA2+S1+S2', 'INI']

In [95]:
# Select the atoms for coavriance matrix calculations
atoms = u.select_atoms('not type H')
atoms_ref = u_ref.select_atoms('not type H')
# Set the number of steps in for coavriance matrix averaging frame (less then the total number of frames)
nsteps = u.trajectory.n_frames 

In [105]:
COV = compute_cov_matrix(atoms, atoms_ref, nsteps)

# Coordination 

In [97]:
coord = coordination(COMB)
df = pd.DataFrame(coord, index=names, columns=names)
df.to_csv('coordination.csv')
df.style.apply(high)

,S1,S2,NA1,NA2,NA1+NA2,NA1+NA2+S1,NA1+NA2+S1+S2,INI
S1,21.2479,0.49315,0.661259,0.297836,0.721375,nan,nan,0.099599
S2,0.254811,26.7113,0.099256,0.062777,0.118384,0.271513,nan,0.0852017
NA1,0.945619,0.484575,7.06902,0.439993,nan,nan,nan,0.102317
NA2,0.724746,0.239114,0.35532,6.93427,nan,nan,nan,0.101591
NA1+NA2,0.743541,0.311329,nan,nan,19.7766,nan,nan,0.0914864
NA1+NA2+S1,nan,0.363703,nan,nan,nan,44.0239,nan,0.0824435
NA1+NA2+S1+S2,nan,nan,nan,nan,nan,nan,91.4122,0.0718618
INI,0.17985,0.166321,0.041907,0.0543982,0.0733052,0.192614,0.23642,7.47379


In [133]:
df = coor_st2_ifs - coor_st2
df.style.apply(highlight_max)

,HP1,HP2,TM5_6,scaff_B,TM3,TM7,TM2,TM8,L1,lipid,L2,scaff
HP1,3.96884,0.0907129,0.101136,0.0437585,0.0695615,0.112006,0.0541285,0.0961448,0.0817569,0.0230307,0.0960172,0.0437585
HP2,0.111664,7.47174,0.0905207,0.0286663,0.10402,0.120934,0.0879592,0.123331,0.122524,0.00309366,0.182571,0.0286663
TM5_6,0.0795849,0.0867123,4.80889,0.0118456,0.0470904,0.079385,0.0361265,0.0894411,0.0607129,0.0269883,0.0799707,0.0118456
scaff_B,0.0380888,0.0347293,0.0551977,2.40317,0.0531213,0.0573824,0.0293414,0.0399506,0.071589,0.0690353,0.0615807,nan
TM3,0.0646655,0.0910423,0.0603847,0.0279853,2.26307,0.0309583,0.0601003,0.0475528,0.07241,0.0463927,0.0853069,0.0279853
TM7,0.106359,0.127895,0.0904579,0.0152423,0.0264346,3.09637,0.0560796,0.111812,0.0823084,0.0307378,0.10847,0.0152423
TM2,0.0291061,0.0463126,0.0670862,0.0718845,0.036997,0.0430636,3.19263,0.0348005,0.0623461,0.0651922,0.0439675,0.0718845
TM8,0.0870002,0.120312,0.148631,0.0344385,0.020741,0.112282,0.0667039,0.855037,0.0783252,0.0443562,0.119974,0.0344385
L1,0.146231,0.119271,0.190299,0.0234482,0.0745955,0.149783,-0.00139828,0.0679127,-2.18377,0.0323342,0.0204202,0.0234482
lipid,0.0522541,0.0241242,0.0889925,0.0935494,0.0561328,0.0472643,0.117986,0.0448965,0.0582063,7.87184,0.0704611,0.0935494


# Mutual information

In [81]:
f = pd.DataFrame(mutual(COMB), index=names, columns=names)
f

Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found
Wrong index format or no elements in the list were found


,S1,S2,NA1,NA2,NA1+NA2,NA1+NA2+S1,NA1+NA2+S1+S2,INI
S1,-343.584899,15.975541,5.966722,5.117223,8.637226,NaN,NaN,8.821380
S2,NaN,-339.873218,9.739698,4.967532,13.633762,21.490246,NaN,9.370566
NA1,NaN,NaN,-147.956759,5.773307,NaN,NaN,NaN,2.945342
NA2,NaN,NaN,NaN,-115.929139,NaN,NaN,NaN,2.535576
NA1+NA2,NaN,NaN,NaN,NaN,-269.659205,NaN,NaN,5.099370
NA1+NA2+S1,NaN,NaN,NaN,NaN,NaN,-526.681887,NaN,11.729041
NA1+NA2+S1+S2,NaN,NaN,NaN,NaN,NaN,NaN,-847.931065,19.299509
INI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-119.682128


# Individual contribution to coordination

In [82]:
for i in range(len(COMB)):
    for j in range(len(COMB)):
        if i<j:
            av_con_coord = average_contribution_coordination(COMB[i], COMB[j])
            # Get residue names for csv
            resnames = [str(k) for k in COMB[i].residues]
            resnames += [str(k) for k in COMB[j].residues]
            resnames = list(dict.fromkeys(resnames))
            ac = pd.DataFrame(av_con_coord, index = resnames, columns = [names[i]+' '+names[j]])
            print(names[i]+' '+names[j]+': \n')
            print(ac)
            ac.to_csv('contribution_coor-'+names[i]+'-'+names[j]+'.csv')

S1 S2: 

                       S1 S2
<Residue LEU, 25>   0.283782
<Residue GLY, 26>   0.224168
<Residue VAL, 104>  0.164442
<Residue TYR, 108>  0.243223
<Residue PHE, 253>  0.169975
<Residue THR, 254>  0.171245
<Residue SER, 256>  0.290743
<Residue PHE, 259>  0.116277
<Residue SER, 355>  0.065226
<Residue ILE, 359>  0.778101
<Residue LEU, 519>  0.562494
<Residue SOD, 517>  0.320333
<Residue LEU, 29>   0.200255
<Residue ARG, 30>   0.050343
<Residue TYR, 107>  0.060364
<Residue ILE, 111>  0.030232
<Residue TRP, 114>  0.211528
<Residue ALA, 319>  0.106793
<Residue PHE, 320>  0.091641
<Residue PHE, 324>  0.092898
<Residue LEU, 400>  0.523652
<Residue ASP, 404>  0.367436
S1 NA1: 

                      S1 NA1
<Residue LEU, 25>   0.216529
<Residue GLY, 26>   0.297371
<Residue VAL, 104>  0.225122
<Residue TYR, 108>  0.266944
<Residue PHE, 253>  0.115791
<Residue THR, 254>  0.281151
<Residue SER, 256>  0.489073
<Residue PHE, 259>  0.572861
<Residue SER, 355>  0.317459
<Residue ILE, 359>  0.13

S2 INI: 

                      S2 INI
<Residue LEU, 29>   0.414031
<Residue ARG, 30>   0.217217
<Residue TYR, 107>  0.613537
<Residue ILE, 111>  0.410788
<Residue TRP, 114>  0.319674
<Residue PHE, 253>  0.320729
<Residue ALA, 319>  0.177631
<Residue PHE, 320>  0.236994
<Residue PHE, 324>  0.131764
<Residue LEU, 400>  0.209242
<Residue ASP, 404>  0.340440
<Residue ARG, 5>    0.387497
<Residue ILE, 187>  0.174331
<Residue SER, 267>  0.288008
<Residue TYR, 268>  0.418549
<Residue GLN, 361>  0.548687
<Residue ASP, 369>  0.420408
NA1 NA2: 

                     NA1 NA2
<Residue ALA, 22>   0.479320
<Residue ASN, 27>   0.736431
<Residue THR, 254>  0.698771
<Residue ASN, 286>  0.604251
<Residue LEU, 519>  0.695586
<Residue SOD, 517>  0.493194
<Residue GLY, 20>   0.329980
<Residue VAL, 23>   0.342000
<Residue ALA, 351>  0.361140
<Residue THR, 354>  0.763567
<Residue SER, 355>  0.276038
<Residue SOD, 516>  0.656130
NA1 NA1+NA2: 

                    NA1 NA1+NA2
<Residue ALA, 22>      0.210897
<

NA1+NA2 INI: 

                    NA1+NA2 INI
<Residue GLY, 20>      0.261642
<Residue ALA, 22>      0.492322
<Residue VAL, 23>      0.527892
<Residue ASN, 27>      0.493548
<Residue THR, 254>     0.437262
<Residue ASN, 286>     0.231888
<Residue ALA, 351>     0.468270
<Residue THR, 354>     0.134868
<Residue SER, 355>     0.298281
<Residue LEU, 519>     0.381376
<Residue SOD, 516>     0.351754
<Residue SOD, 517>     0.557053
<Residue ARG, 5>       0.558632
<Residue ILE, 187>     0.414567
<Residue SER, 267>     0.317032
<Residue TYR, 268>     0.583400
<Residue GLN, 361>     0.355736
<Residue ASP, 369>     0.721477
NA1+NA2+S1 NA1+NA2+S1+S2: 

                    NA1+NA2+S1 NA1+NA2+S1+S2
<Residue GLY, 20>                   0.169881
<Residue ALA, 22>                   0.237532
<Residue VAL, 23>                   0.252307
<Residue LEU, 25>                   0.253953
<Residue GLY, 26>                   0.219912
<Residue ASN, 27>                   0.250896
<Residue VAL, 104>                

# Coordination channel

In [98]:
# Calculate per-residue coordination contribution to a channel between A and B sites
A = COMB[0]
B = COMB[1]
channel = coordination_channel(A, B)
channel.shape

(516,)

In [99]:
# Create beta column for the pdb file
betas = np.empty(u.trajectory.n_atoms)
for i in range(channel.shape[0]):
    atom1 = int(str(u.residues[i].atoms[0]).split(':')[0].split(' ')[1])
    atom_1 = int(str(u.residues[i].atoms[-1]).split(':')[0].split(' ')[1])
    betas[atom1:atom_1+1] =channel[i]
betas = np.round_(betas, 2)


In [100]:
# Use the ref file pdb as a templete to write coordination-channel pdb
pdb = open(u_ref.trajectory.filename, 'rt')
pdb_out = open('channel-'+names[1]+'_'+names[2]+'.pdb', 'wt')
i = 0
for line in pdb:
    if 'ATOM' in line:
        if betas[i]<10:
            pdb_out.write(line.replace('0.00', "{:0.2f}".format(betas[i])))
        else:
            pdb_out.write(line.replace(' 0.00', "{:0.2f}".format(betas[i])))
        i+=1
pdb.close()
pdb_out.close()
